# Linear Regression Consulting Project

Congratulations! You've been contracted by Hyundai Heavy Industries to help them build a predictive model for some ships. [Hyundai Heavy Industries](http://www.hyundai.eu/en) is one of the world's largest ship manufacturing companies and builds cruise liners.

You've been flown to their headquarters in Ulsan, South Korea to help them give accurate estimates of how many crew members a ship will require.

They are currently building new ships for some customers and want you to create a model and use it to predict how many crew members the ships will need.

Here is what the data looks like so far:

    Description: Measurements of ship size, capacity, crew, and age for 158 cruise
    ships.


    Variables/Columns
    Ship Name     1-20
    Cruise Line   21-40
    Age (as of 2013)   46-48
    Tonnage (1000s of tons)   50-56
    passengers (100s)   58-64
    Length (100s of feet)  66-72
    Cabins  (100s)   74-80
    Passenger Density   82-88
    Crew  (100s)   90-96
    
It is saved in a csv file for you called "cruise_ship_info.csv". Your job is to create a regression model that will help predict how many crew members will be needed for future ships. The client also mentioned that they have found that particular cruise lines will differ in acceptable crew counts, so it is most likely an important feature to include in your analysis! 

Once you've created the model and tested it for a quick check on how well you can expect it to perform, make sure you take a look at why it performs so well!

In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName('Hyundai').getOrCreate()

In [5]:
 data = spark.read.csv('/FileStore/tables/cruise_ship_info.csv', inferSchema=True, header=True)

In [6]:
data.toPandas().head()

,Ship_name,Cruise_line,Age,Tonnage,passengers,length,cabins,passenger_density,crew
0,Journey,Azamara,6,30.277,6.94,5.94,3.55,42.64,3.55
1,Quest,Azamara,6,30.277,6.94,5.94,3.55,42.64,3.55
2,Celebration,Carnival,26,47.262,14.86,7.22,7.43,31.80,6.70
3,Conquest,Carnival,11,110.000,29.74,9.53,14.88,36.99,19.10
4,Destiny,Carnival,17,101.353,26.42,8.92,13.21,38.36,10.00


In [7]:
data.printSchema()

root
-- Ship_name: string (nullable = true)
-- Cruise_line: string (nullable = true)
-- Age: integer (nullable = true)
-- Tonnage: double (nullable = true)
-- passengers: double (nullable = true)
-- length: double (nullable = true)
-- cabins: double (nullable = true)
-- passenger_density: double (nullable = true)
-- crew: double (nullable = true)

In [8]:
#Para el modelo usaremos todas las features excepto el nombre del barco
#Añadiremos una columna con los valores de la columna Cruise_line pero en formato String

from pyspark.ml.feature import StringIndexer

In [9]:
indexer = StringIndexer(inputCol='Cruise_line', outputCol='Cruise_line_index')
data2 = indexer.fit(data).transform(data)

In [10]:
data2.toPandas().head()

,Ship_name,Cruise_line,Age,Tonnage,passengers,length,cabins,passenger_density,crew,Cruise_line_index
0,Journey,Azamara,6,30.277,6.94,5.94,3.55,42.64,3.55,16.0
1,Quest,Azamara,6,30.277,6.94,5.94,3.55,42.64,3.55,16.0
2,Celebration,Carnival,26,47.262,14.86,7.22,7.43,31.80,6.70,1.0
3,Conquest,Carnival,11,110.000,29.74,9.53,14.88,36.99,19.10,1.0
4,Destiny,Carnival,17,101.353,26.42,8.92,13.21,38.36,10.00,1.0


In [11]:
#Importamos paquetes para arreglar los datos y dejarlos en el formato que la librería ml acepta
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [12]:
data2.columns

Out[11]: ['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'Cruise_line_index']

In [13]:
#Creamos objeto assembler
assembler = VectorAssembler(inputCols=['Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'Cruise_line_index'], outputCol='features')

In [14]:
#Transformamos los datos
output = assembler.transform(data2)

In [15]:
output.printSchema()

root
-- Ship_name: string (nullable = true)
-- Cruise_line: string (nullable = true)
-- Age: integer (nullable = true)
-- Tonnage: double (nullable = true)
-- passengers: double (nullable = true)
-- length: double (nullable = true)
-- cabins: double (nullable = true)
-- passenger_density: double (nullable = true)
-- crew: double (nullable = true)
-- Cruise_line_index: double (nullable = false)
-- features: vector (nullable = true)

In [16]:
#Seleccionamos las columnas para el dataset final para el modelo
final_data = output.select('features','crew')

In [17]:
final_data.toPandas().head()

,features,crew
0,"[6.0, 30.276999999999997, 6.94, 5.94, 3.55, 42...",3.55
1,"[6.0, 30.276999999999997, 6.94, 5.94, 3.55, 42...",3.55
2,"[26.0, 47.262, 14.86, 7.22, 7.43, 31.8, 1.0]",6.70
3,"[11.0, 110.0, 29.74, 9.53, 14.88, 36.99, 1.0]",19.10
4,"[17.0, 101.353, 26.42, 8.92, 13.21, 38.36, 1.0]",10.00


In [18]:
#División en test y training set
train_data, test_data = final_data.randomSplit([0.7,0.3]) 

In [19]:
train_data.describe().toPandas().head()

,summary,crew
0,count,117
1,mean,7.719743589743601
2,stddev,3.2893158972069
3,min,0.59
4,max,19.1


In [20]:
test_data.describe().toPandas().head()

,summary,crew
0,count,41
1,mean,8.00658536585366
2,stddev,4.091186019821209
3,min,0.59
4,max,21.0


In [21]:
from pyspark.ml.regression import LinearRegression

In [22]:
#Creamos objeto LinearRegression
lr = LinearRegression(labelCol='crew')

In [23]:
lr_model = lr.fit(train_data)

In [24]:
test_results = lr_model.evaluate(test_data)

In [25]:
test_results.residuals.toPandas().head(20)

,residuals
0,-0.279269
1,-1.553690
2,-0.827410
3,-1.297410
4,0.514376
5,-1.334432
6,0.922371
7,0.820704
8,-0.707821
9,1.014193


In [26]:
test_results.r2

Out[26]: 0.961378097109634

In [27]:
#Error cuadrático medio
test_results.rootMeanSquaredError

Out[27]: 0.7941529028594377

In [28]:
final_data.describe().toPandas().head()

,summary,crew
0,count,158
1,mean,7.794177215189873
2,stddev,3.503486564627034
3,min,0.59
4,max,21.0


Vemos que la media de número de tripulantes es 7.79, con una desviación estándar de 3.50, nuestra RMSE es 0.75, considerando los valores anteriores nuestro modelo parece ser bueno.

Veremos qué tanta correlación hay entre las features predictoras y la feature a predecir, así veremos si los resultados del modelo tienen sentido o son solo una coincidencia

In [31]:
from pyspark.sql.functions import corr

In [32]:
data.select(corr('crew','passengers')).toPandas().head()

,"corr(crew, passengers)"
0,0.915234


Vemos que las columnas crew y passengers tienen una alta correlación. Intuitivamente esto hace sentido, ya que mientras más pasajeros haya a bordo debiera necesitarse un mayor número en la tripulación.

In [34]:
data.select(corr('crew','cabins')).toPandas().head()

,"corr(crew, cabins)"
0,0.950823
